# Neural Network with 2 hidden layers

### Load Modules

In [1]:
# Set the Random Seed
seed_value= 2022

import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import numpy  as np
import pandas  as pd
import datetime
import random
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.formula.api as smf
import tensorflow as tf
from keras.models     import Sequential, load_model
from keras.layers     import Activation, Dense, LeakyReLU
from keras.optimizers import Adam
from keras.callbacks  import EarlyStopping, ModelCheckpoint
from keras.regularizers import l1
from tensorflow.keras.losses import MeanSquaredError

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [3]:
def reset_seeds(seed_value=2022):
    os.environ['PYTHONHASHSEED']=str(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)
reset_seeds()

### Load Dataset - only traditional features

In [4]:
# Load monthly firm characteristics raw data
df = pd.read_parquet('C:/Users/rafae/Documents/HSG/Master Thesis/Data/Final/data07_model_input.parquet')
df = df.sort_values(by=['YM', 'permno'])
df = df.set_index(['year', 'YM', 'permno'])
df

gvkey  reprisk_id         prc         vol         mve_m  \
year YM      permno                                                             
2007 2007-01 10025    11903       37172   45.320000      8086.0  3.700557e+05   
             10026    12825       12684   39.689999      7613.0  7.653725e+05   
             10042    12139        4832    0.720000     26008.0  3.598898e+04   
             10078    12136        1719    6.130000  11333293.0  2.390900e+07   
             10104    12142        4413   16.430000   7234361.0  8.892640e+07   
...                     ...         ...         ...         ...           ...   
2021 2021-12 93304   184167       91339   36.750000    183303.0  1.695898e+06   
             93373   184323       74074    3.020000    528712.0  2.065325e+05   
             93374   184899       64442   74.510002    159495.0  8.587073e+06   
             93423    10567       22547   39.490002    254917.0  3.661156e+06   
             93436   184996       24952  936.719971   6361641.0  1.092218e+09   

                       absacc       acc    aeavol       age       agr  ...  \
year YM      permno                                                    ...   
2007 2007-01 10025   0.698728 -0.745547 -0.646819  0.457506 -0.979644  ...   
             10026   0.577608 -0.635623 -0.393384  0.457506  0.118575  ...   
             10042   0.990840 -0.989822 -0.894148  0.457506 -0.989822  ...   
             10078   0.654962 -0.711959 -0.128753  0.905344 -0.147074  ...   
             10104  -0.014758 -0.107379  0.780153  0.905344  0.770992  ...   
...                       ...       ...       ...       ...       ...  ...   
2021 2021-12 93304  -0.855522  0.706269 -0.871045 -0.500896  0.613134  ...   
             93373  -0.213134  0.143881  0.663284 -0.500896 -0.875821  ...   
             93374  -0.514030  0.423284 -0.223881 -0.500896  0.328955  ...   
             93423   0.242985 -0.303881  0.560597  0.143881 -0.584478  ...   
             93436   0.584478 -0.620299 -0.362388 -0.500896  0.856716  ...   

                     sic2_73  sic2_75  sic2_78  sic2_79  sic2_80  sic2_81  \
year YM      permno                                                         
2007 2007-01 10025         0        0        0        0        0        0   
             10026         0        0        0        0        0        0   
             10042         0        0        0        0        0        0   
             10078         0        0        0        0        0        0   
             10104         1        0        0        0        0        0   
...                      ...      ...      ...      ...      ...      ...   
2021 2021-12 93304         0        0        0        0        0        0   
             93373         0        0        0        0        0        0   
             93374         0        0        0        0        0        0   
             93423         0        0        0        1        0        0   
             93436         0        0        0        0        0        0   

                     sic2_82  sic2_83  sic2_87  sic2_99  
year YM      permno                                      
2007 2007-01 10025         0        0        0        0  
             10026         0        0        0        0  
             10042         0        0        0        0  
             10078         0        0        0        0  
             10104         0        0        0        0  
...                      ...      ...      ...      ...  
2021 2021-12 93304         0        0        0        0  
             93373         0        0        0        0  
             93374         0        0        0        0  
             93423         0        0        0        0  
             93436         0        0        0        0  

[338288 rows x 188 columns]

In [5]:
# Select only relevant columns for X and Y (exclude industry & reprisk rating dummy for now)
info_vars = ['YM', 'year', 'permno', 'gvkey', 'reprisk_id', 'prc', 'vol', 'mve_m']
X_vars = ['absacc', 'acc', 'aeavol', 'age', 'agr', 'baspread', 'beta', 'betasq', 'bm', 'bm_ia', 'cash', 'cashdebt',
          'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv', 'chmom', 'chpmia', 'chtx', 'cinvest',
          'convind', 'currat', 'depr', 'divi', 'divo', 'dolvol', 'dy', 'ear', 'egr', 'ep', 'gma', 'grcapx', 'grltnoa',
          'herf', 'hire', 'idiovol', 'ill', 'indmom', 'invest', 'lev', 'lgr', 'maxret', 'mom12m', 'mom1m', 'mom36m',
          'mom6m', 'ms', 'mve', 'mve_ia', 'nincr', 'operprof', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr',
          'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc',
          'pricedelay', 'ps', 'quick', 'rd', 'rd_mve', 'rd_sale', 'realestate', 'retvol', 'roaq', 'roavol', 'roeq',
          'roic', 'rsup', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp', 'std_dolvol',
          'std_turn', 'stdacc', 'stdcf', 'tang', 'tb', 'turn', 'zerotrade']
sic2_vars = [col for col in df if col.startswith('sic2')]
reprisk_vars = ['country_sector_average', 'country_sector_average_01', 'current_rri', 'current_rri_01',
                'peak_rri', 'peak_rri_01', 'trend_rri', 'trend_rri_01']
reprisk_rating_vars = [col for col in df if col.startswith('reprisk_rating')]
Y_vars = ['ret', 'ret_wins', 'ret_ex']

# X
X = df[X_vars + sic2_vars]

# Y
Y = df[['ret_ex']]

X.shape, Y.shape

((338288, 162), (338288, 1))

### Hyperparameter Optimization: 4-Fold CV (12y/4 = 3y) and 3y Test

In [6]:
# Training (12y - 80%) and Test set (3y - 20%)
X_trai = X.loc['2007':'2018']
Y_trai = Y.loc['2007':'2018']

X_test = X.loc['2019':'2021']
Y_test = Y.loc['2019':'2021']

In [7]:
# 4-Fold cross validation (9y training and 3y validation)
K_FOLDs = 4
YEARS = list(X_trai.index.unique(level='year').astype('str')) 
TOT = len(YEARS)
TRA = int(TOT* (K_FOLDs-1) / K_FOLDs)
OFF = TOT - TRA

for FOLD in range(K_FOLDs):
    VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
    TRAI = [x for x in YEARS if x not in VALI]
    print(VALI, TRAI)    

['2007', '2008', '2009'] ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
['2010', '2011', '2012'] ['2007', '2008', '2009', '2013', '2014', '2015', '2016', '2017', '2018']
['2013', '2014', '2015'] ['2007', '2008', '2009', '2010', '2011', '2012', '2016', '2017', '2018']
['2016', '2017', '2018'] ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']


In [8]:
# Model
model_name = 'NN2_trad'

def create_NN2(l_rate=0.01, l1_pen=0.0001):
    model = Sequential()
    model.add(Dense(32,
                    activation='relu',
                    input_dim=X_trai.shape[1],
                    kernel_regularizer=l1(l1_pen)))
    model.add(Dense(16,
                    activation='relu',
                    kernel_regularizer=l1(l1_pen)))
    model.add(Dense(1))
    model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=l_rate))
    return model

In [9]:
# Hyperparameter optimization
Y_val_preds = Y_trai.copy()
Y_test_preds = Y_test.copy()
results = []

earlyStopping  = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')

model_ix = 0
for l1_pen in [0.001, 0.0005, 0.0001, 0.00005, 0.00001]:
        for l_rate in [0.01, 0.005, 0.001]:
            for ensemble in list(range(0,10)):
                print("•", model_ix, ':',  end=' ')
                    
                for FOLD in range(K_FOLDs):
                    VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
                    TRAI = [x for x in YEARS if x not in VALI]
                        
                    mcp_save = ModelCheckpoint(
                        '%s/%s_%d_%d.h5'%(model_name, model_name, model_ix, FOLD),
                        save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True
                    )

                    reset_seeds(ensemble*10000)
                        
                    NN_model = create_NN2(l_rate, l1_pen)
                        
                    history = NN_model.fit(
                        X_trai.loc[TRAI], Y_trai.loc[TRAI],
                        validation_data=(X_trai.loc[VALI], Y_trai.loc[VALI]),
                        epochs=200, verbose=0, shuffle=True, batch_size=2000,
                        callbacks=[earlyStopping, mcp_save]
                    )
                    tra_loss = np.min(history.history['loss'])
                    val_loss = np.min(history.history['val_loss'])

                    results.append({
                        'model_ix' :model_ix,
                        'l_rate'   :l_rate,
                        'l1_pen'   :l1_pen,
                        'fold'     :FOLD,
                        'tra_loss' :tra_loss,
                        'val_loss' :val_loss
                    })
                    
                    # Calculate validation predictions
                    best_model = create_NN2(l_rate, l1_pen)
                    best_model.load_weights(r'%s/%s_%d_%d.h5'%(model_name, model_name, model_ix, FOLD));
                    Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])
                    
                    # Calculate predictions for test data, if FOLD = 0
                    if FOLD==0:
                        Y_test_preds.loc[['2019','2020', '2021'], '%s_%d'%(model_name, model_ix)] = best_model.predict(X_test)

                model_ix += 1

• 0 : Epoch 8: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 8: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 8: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 10: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 1 : Epoch 7: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 14: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 12: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 2 : Epoch 8: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 3 : E

1929/1929 [==============================] - 4s 2ms/step
Epoch 15: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 14: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 13: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 22 : Epoch 17: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 16: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 16: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 17: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 23 : Epoch 20: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 21: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 18: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 23: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 24 : Epoch 22: early stop

1929/1929 [==============================] - 4s 2ms/step
Epoch 9: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 9: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 11: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 43 : Epoch 10: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 15: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 10: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 11: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 44 : Epoch 15: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 11: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 12: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 12: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 45 : Epoch 12: early stoppi

• 63 : Epoch 14: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 15: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 13: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 13: early stopping
2066/2066 [==============================] - 5s 2ms/step
• 64 : Epoch 15: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 15: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 14: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 16: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 65 : Epoch 22: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 18: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 16: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 15: early stopping
2066/2066 [==============================] - 4s 2ms/

C:\Users\rafae\AppData\Local\Temp\ipykernel_4152\2486219011.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])


...

Epoch 32: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 33: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 28: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 149 : Epoch 65: early stopping
   1/1929 [..............................] - ETA: 30s

C:\Users\rafae\AppData\Local\Temp\ipykernel_4152\2486219011.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])


1929/1929 [==============================] - 4s 2ms/step


C:\Users\rafae\AppData\Local\Temp\ipykernel_4152\2486219011.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_test_preds.loc[['2019','2020', '2021'], '%s_%d'%(model_name, model_ix)] = best_model.predict(X_test)


Epoch 55: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 45: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 48: early stopping
2066/2066 [==============================] - 4s 2ms/step


In [10]:
# Save Y_val_preds
Y_val_preds.to_csv(r'%s/%s_val_preds.csv'%(model_name, model_name))
Y_val_preds

ret_ex  NN2_trad_0  NN2_trad_1  NN2_trad_2  NN2_trad_3  \
year YM      permno                                                             
2007 2007-01 10025  -0.031894    0.011511    0.010265    0.011895    0.007963   
             10026  -0.042317    0.011511    0.010265    0.011895    0.007963   
             10042  -0.125751    0.011511    0.010265    0.011895    0.007963   
             10078  -0.080607    0.011511    0.010265    0.011895    0.007963   
             10104  -0.046341    0.011511    0.010265    0.011895    0.007963   
...                       ...         ...         ...         ...         ...   
2018 2018-12 93420   0.086508    0.007405    0.004575    0.010017    0.004956   
             93422   0.466817    0.007405    0.004575    0.010017    0.004956   
             93423   0.105036    0.007405    0.004575    0.010017    0.004956   
             93429  -0.048712    0.007405    0.004575    0.010017    0.004956   
             93436  -0.079564    0.007405    0.004575    0.010017    0.004956   

                     NN2_trad_4  NN2_trad_5  NN2_trad_6  NN2_trad_7  \
year YM      permno                                                   
2007 2007-01 10025     0.010351    0.010074    0.009212    0.008278   
             10026     0.010351    0.010073    0.009212    0.008278   
             10042     0.010351    0.010069    0.009212    0.008278   
             10078     0.010351    0.010073    0.009212    0.008277   
             10104     0.010351    0.010074    0.009212    0.008277   
...                         ...         ...         ...         ...   
2018 2018-12 93420     0.006808    0.007098    0.006236    0.005537   
             93422     0.006808    0.007098    0.006236    0.005537   
             93423     0.006808    0.007098    0.006236    0.005537   
             93429     0.006808    0.007098    0.006236    0.005537   
             93436     0.006808    0.007098    0.006236    0.005537   

                     NN2_trad_8  ...  NN2_trad_140  NN2_trad_141  \
year YM      permno              ...                               
2007 2007-01 10025     0.004905  ...      0.014709      0.013735   
             10026     0.004897  ...      0.013402      0.011478   
             10042     0.004893  ...     -0.023655     -0.046739   
             10078     0.004895  ...      0.015101      0.006162   
             10104     0.004893  ...      0.014709      0.009012   
...                         ...  ...           ...           ...   
2018 2018-12 93420     0.004084  ...     -0.047416      0.010884   
             93422     0.004084  ...     -0.060749      0.006642   
             93423     0.004084  ...      0.008266     -0.000921   
             93429     0.004084  ...      0.011372      0.011003   
             93436     0.004084  ...      0.009355     -0.003701   

                     NN2_trad_142  NN2_trad_143  NN2_trad_144  NN2_trad_145  \
year YM      permno                                                           
2007 2007-01 10025       0.015137      0.017624      0.025608      0.031536   
             10026       0.011170      0.009705      0.010831      0.013463   
             10042      -0.020375     -0.009305     -0.013054     -0.010110   
             10078       0.007734      0.012032      0.003991      0.016626   
             10104       0.007695      0.013807      0.017185      0.009850   
...                           ...           ...           ...           ...   
2018 2018-12 93420      -0.015578     -0.041345      0.007947     -0.024384   
             93422      -0.041883     -0.012960     -0.005954     -0.026264   
             93423       0.005983      0.010616     -0.002082      0.005450   
             93429       0.004800      0.013533      0.005680      0.012094   
             93436       0.003847      0.009476      0.000207      0.012663   

                     NN2_trad_146  NN2_trad_147  NN2_trad_148  NN2_trad_149  
year YM      permno                                   

In [11]:
# Save Y_test_preds
Y_test_preds.to_csv(r'%s/%s_test_preds.csv'%(model_name, model_name))
Y_test_preds

ret_ex  NN2_trad_0  NN2_trad_1  NN2_trad_2  NN2_trad_3  \
year YM      permno                                                             
2019 2019-01 10026   0.004225    0.011511    0.010265    0.011895    0.007963   
             10104   0.036026    0.011511    0.010265    0.011895    0.007963   
             10107   0.075381    0.011511    0.010265    0.011895    0.007963   
             10138   0.072777    0.011511    0.010265    0.011895    0.007963   
             10145   0.076596    0.011511    0.010265    0.011895    0.007963   
...                       ...         ...         ...         ...         ...   
2021 2021-12 93304  -0.096386    0.011511    0.010265    0.011895    0.007963   
             93373  -0.019481    0.011511    0.010265    0.011895    0.007963   
             93374  -0.047552    0.011511    0.010265    0.011895    0.007963   
             93423  -0.072569    0.011511    0.010265    0.011895    0.007963   
             93436  -0.113609    0.011511    0.010265    0.011895    0.007963   

                     NN2_trad_4  NN2_trad_5  NN2_trad_6  NN2_trad_7  \
year YM      permno                                                   
2019 2019-01 10026     0.010351    0.010074    0.009212    0.008279   
             10104     0.010351    0.010071    0.009212    0.008278   
             10107     0.010351    0.010074    0.009212    0.008278   
             10138     0.010351    0.010073    0.009212    0.008278   
             10145     0.010351    0.010073    0.009212    0.008278   
...                         ...         ...         ...         ...   
2021 2021-12 93304     0.010351    0.010072    0.009212    0.008277   
             93373     0.010351    0.010075    0.009212    0.008277   
             93374     0.010351    0.010072    0.009212    0.008277   
             93423     0.010351    0.010071    0.009212    0.008277   
             93436     0.010351    0.010073    0.009212    0.008277   

                     NN2_trad_8  ...  NN2_trad_140  NN2_trad_141  \
year YM      permno              ...                               
2019 2019-01 10026     0.004895  ...      0.011137      0.009537   
             10104     0.004906  ...      0.001408      0.010083   
             10107     0.004896  ...      0.014709      0.008990   
             10138     0.004895  ...      0.013777      0.009500   
             10145     0.004900  ...      0.009680      0.008903   
...                         ...  ...           ...           ...   
2021 2021-12 93304     0.004895  ...      0.012452      0.010854   
             93373     0.004893  ...     -0.009344     -0.007708   
             93374     0.004903  ...      0.009387      0.008697   
             93423     0.004892  ...      0.011000     -0.000045   
             93436     0.004890  ...      0.007021      0.002086   

                     NN2_trad_142  NN2_trad_143  NN2_trad_144  NN2_trad_145  \
year YM      permno                                                           
2019 2019-01 10026       0.008004      0.006528      0.011920      0.006303   
             10104       0.007149      0.014437      0.005109      0.008852   
             10107       0.008857      0.010166      0.014434      0.006693   
             10138       0.005389      0.009839      0.011254      0.007971   
             10145       0.007258      0.008701      0.013430      0.011732   
...                           ...           ...           ...           ...   
2021 2021-12 93304       0.009769      0.014506      0.013393      0.007730   
             93373      -0.005936      0.005287      0.003187      0.002504   
             93374       0.006769      0.010154      0.014935      0.006744   
             93423       0.009666      0.023509      0.015797      0.022272   
             93436      -0.017336      0.004759      0.005494      0.003285   

                     NN2_trad_146  NN2_trad_147  NN2_trad_148  NN2_trad_149  
year YM      permno                                   

In [12]:
# Result overview
table = pd.DataFrame(results)
table = table.groupby(['model_ix', 'l_rate', 'l1_pen']).mean().sort_values('val_loss')
table.to_csv(r'%s/%s_results.csv'%(model_name, model_name))

np.sqrt(table[['tra_loss', 'val_loss']].head(20)) * 100

,,,tra_loss,val_loss
model_ix,l_rate,l1_pen,,
81,0.001,0.00010,12.989207,12.954304
85,0.001,0.00010,12.989086,12.956401
89,0.001,0.00010,12.988984,12.957620
112,0.001,0.00005,12.974317,12.958335
84,0.001,0.00010,12.990980,12.961921
87,0.001,0.00010,12.990828,12.962749
113,0.001,0.00005,12.971720,12.965642
121,0.010,0.00001,12.981387,12.965732
117,0.001,0.00005,12.977291,12.967521


## Model Selection: Maximize SR on LS-Portfolio on validation data

In [13]:
# Model
model_name = 'NN2_trad'

# Get predictions
val = pd.read_csv(r'%s/%s_val_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])
test = pd.read_csv(r'%s/%s_test_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])

# Get overview table
table = pd.read_csv(r'%s/%s_results.csv'%(model_name, model_name)).sort_values(['model_ix'])

# Extract hyperparameters
list_l_rate = list(pd.unique(table['l_rate']))
list_l1_pen = list(pd.unique(table['l1_pen']))

In [14]:
# Loop over hyperparameter combinations to get ensemble predictions for each hyperparameter combination
val_results = []
hyper_comb = 0
for l_rate in list_l_rate:
    for l1_pen in list_l1_pen:
        # Get list of relevant model_ix
        list_model_ix = table[(table['l_rate'] == l_rate) & (table['l1_pen'] == l1_pen)]
        list_model_ix = list(list_model_ix['model_ix'])
        # Get list of column names
        list_col_names = []
        for ele in list_model_ix:
            list_col_names.append('%s_%d'%(model_name, ele))
        # Select relevant returns and average predictions
        val_ret = val[list_col_names]
        val_ret = val_ret.mean(axis=1)
        # Append ensemble return prediction to true returns
        comb = val[['ret_ex']].copy()
        comb['ret_pred'] = val_ret
        comb = comb.reset_index()
        # Sort the data by predicted returns and divide the data into quintiles
        comb['quintile'] = comb.groupby(['YM'])['ret_pred'].transform(lambda x: pd.qcut(x.rank(method='first'), 5, labels=np.arange(1,6)))
        # Calculate the mean return for each YM
        comb_mean = comb.groupby(['YM', 'quintile']).agg(ret_ex = ('ret_ex', 'mean'))
        # Add LS-Strategy
        comb_mean = comb_mean[['ret_ex']].unstack().add_prefix('Q')
        comb_mean.columns = comb_mean.columns.droplevel(0)
        comb_mean.columns.name = None
        comb_mean['LS'] = comb_mean['Q5'] - comb_mean['Q1']
        # Calculate the average return, standard deviation and Sharpe Ratio (annualized) per Quintile
        summary = pd.DataFrame()
        summary['mean'] = comb_mean.mean()
        summary['std'] = comb_mean.std()
        summary['SR'] = summary['mean'] / summary['std'] * np.sqrt(12)
        # Append results
        val_results.append({
            'hyper_comb':hyper_comb,
            'l_rate'    :l_rate,
            'l1_pen'    :l1_pen,
            'SR_Q1'     :summary.loc['Q1','SR'],
            'SR_Q5'     :summary.loc['Q5','SR'],
            'SR_LS'     :summary.loc['LS','SR'],
            'Mean_Q1'   :summary.loc['Q1','mean'],
            'Mean_Q5'   :summary.loc['Q5','mean'],
            'Mean_LS'   :summary.loc['LS','mean'],
        })
        hyper_comb += 1
        
# Save results
val_results = pd.DataFrame(val_results)
val_results.to_csv(r'results/{}_val.csv'.format(model_name))

In [15]:
# Select best hyperparemeters (max. SR_LS)
val_results = val = pd.read_csv(r'results/{}_val.csv'.format(model_name))
val_results = val_results.set_index(['l_rate', 'l1_pen']).sort_values(['SR_LS'], ascending=False)
l_rate_opt, l1_pen_opt = tuple([x for x in val_results.iloc[0].name[0:]])
print('Optimal l_rate:', l_rate_opt)
print('Optimal l1_pen:', l1_pen_opt)
val_results

Optimal l_rate: 0.001
Optimal l1_pen: 1e-05


Unnamed: 0  hyper_comb     SR_Q1     SR_Q5     SR_LS  \
l_rate l1_pen                                                          
0.001  0.00001          14          14  0.088190  0.442484  0.820030   
0.005  0.00001           9           9  0.059802  0.421488  0.616700   
0.001  0.00005          13          13  0.033631  0.557060  0.515190   
0.005  0.00005           8           8  0.055401  0.621244  0.507178   
0.010  0.00005           3           3  0.057433  0.592869  0.446298   
       0.00001           4           4  0.058926  0.363742  0.419169   
0.001  0.00050          11          11  0.261740  0.461560  0.411561   
0.010  0.00050           1           1  0.245685  0.392133  0.399862   
0.005  0.00010           7           7  0.105644  0.616552  0.353338   
0.001  0.00010          12          12  0.127254  0.571299  0.334417   
0.010  0.00100           0           0  0.266001  0.396349  0.266297   
0.005  0.00100           5           5  0.324262  0.375848  0.106207   
0.010  0.00010           2           2  0.258932  0.313652 -0.102285   
0.001  0.00100          10          10  0.397396  0.287942 -0.184493   
0.005  0.00050           6           6  0.376240  0.271702 -0.313725   

                 Mean_Q1   Mean_Q5   Mean_LS  
l_rate l1_pen                                 
0.001  0.00001  0.001927  0.008234  0.006307  
0.005  0.00001  0.001384  0.007287  0.005904  
0.001  0.00005  0.000816  0.008111  0.007295  
0.005  0.00005  0.001359  0.008665  0.007306  
0.010  0.00005  0.001404  0.008062  0.006658  
       0.00001  0.001406  0.005966  0.004560  
0.001  0.00050  0.004789  0.007579  0.002790  
0.010  0.00050  0.004452  0.006938  0.002486  
0.005  0.00010  0.002522  0.007638  0.005117  
0.001  0.00010  0.003081  0.007830  0.004749  
0.010  0.00100  0.004881  0.006589  0.001708  
0.005  0.00100  0.005938  0.006722  0.000784  
0.010  0.00010  0.005879  0.004549 -0.001330  
0.001  0.00100  0.006763  0.005424 -0.001339  
0.005  0.00050  0.007681  0.004563 -0.003118

## Out-of-Sample: Get Predictions of model with best hyperparameter combination

In [16]:
# Get list of model_ix's with best hyperparameter combinations and make ensemble predictions
list_model_ix = table[(table['l_rate'] == l_rate_opt) & (table['l1_pen'] == l1_pen_opt)]
list_model_ix = list(list_model_ix['model_ix'])

# Get list of column names
list_col_names = []
for ele in list_model_ix:
    list_col_names.append('%s_%d'%(model_name, ele))
    
# Select relevant returns and average predictions
test_ret = test[list_col_names].copy()
test_ret['y_pred'] = test_ret.mean(axis=1)

# Save predictions
test_ret = test_ret.reset_index()
test_ret = test_ret[['YM', 'permno', 'y_pred']]
test_ret.to_csv(r'results/{}_predictions.csv'.format(model_name), index=False)